# Generating Power Station Maps

In [1]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

#!pip install folium
import folium # map rendering library
import re
print('Libraries imported.')

Libraries imported.


## Data Collection and Preparation

Reading in data

In [2]:
power_df = pd.read_excel('/content/drive/My Drive/Colab Data/ebvc_routes/PowerStation1.xlsx' )

Next up we modify the null values in the dataframe

A function is then created to read in latitude and longitude values from different forms of data utilizing regular expressions.

In [3]:
def converter(x):
  conv = lambda c: float(c) if "." in c else int(c)
  digits = [conv(i) for i in re.findall(r"\d{1,3}\.\d+|\d{1,3}", x)]
  if len(digits)==6:
    a,b,c,d,e,f = digits
    lat = a + b/60 + c/3600
    lng = d + e/60 + f/3600
  elif len(digits) == 4:
    a,b,c,d = digits
    lat = a + b/60
    lng = c + d/60
  elif len(digits) == 2:
    lat,lng = digits
  if any(b in x for b in ["W","-"]):
    lng*=-1

  return lat,lng

In [4]:
power_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           624 non-null    object
 1   Site           451 non-null    object
 2   Coordinates    600 non-null    object
 3   Capacity (MW)  606 non-null    object
dtypes: object(4)
memory usage: 20.5+ KB


The data then gets filtered by datatype and the shape is printed in the next cell.

In [5]:
coord_df = power_df.iloc[power_df.index[[type(s)==str for s in power_df['Coordinates']]],:]

In [6]:
coord_df.shape

(600, 4)

The created function (some 3 cells above) is then used to covert latitude and longitude values.

In [ ]:
latitude, longitude = [],[]
for i in coord_df['Coordinates']:
  la,ln = converter(i)
  latitude.append(la)
  longitude.append(ln)
coord_df['latitude'] = latitude
coord_df["longitude"] = longitude

A part of the data is visualized.

In [8]:
coord_df.head()

,Name,Site,Coordinates,Capacity (MW),latitude,longitude
0,Forsmark 1,Forsmark,60°24′20″N 18°09′41″E,984,60.405556,18.161389
1,Forsmark 2,Forsmark,60°24′14″N 18°10′24″E,1120,60.403889,18.173333
2,Forsmark 3,Forsmark,60°24′10″N 18°10′31″E,1167,60.402778,18.175278
3,Oskarshamn 3,Figeholm,57°24′58″N 16°40′23″E,1400,57.416111,16.673056
4,Ringhals 1,Gloppe,57°15′43″N 12°06′41″E,865,57.261944,12.111389


The map is initialized

In [13]:
map_stations = folium.Map(location = [51.5074,-0.1278],
                          tiles='Stamen Terrain',
                          zoom_start=6)
map_stations.add_child(folium.LatLngPopup());

Finally we loop through power stations and insert popups for easy use of map.

In [14]:
for name, site, cap, lat, lon in coord_df[['Name', 'Site', 'Capacity (MW)', 'latitude','longitude']].values:
    poppin = "{0}\n{1}\n{2}".format(
        name if name != np.nan else "",
        site if site != np.nan else "",
         str(cap) + " MW" if cap != np.nan else "")
    label = folium.Popup(poppin,parse_html=True, max_width="20%")
    folium.Marker( 
        [lat, lon],
        popup=label,
         icon=folium.Icon(color='green'),
        ).add_to(map_stations)
        
map_stations

The map and dataframe are saved as html and csv files respectively.

In [ ]:
map_stations.save("/content/drive/My Drive/Colab Data/ebvc_routes/Power stations and plants/PowerPlants_Openstreetmap.html")

In [ ]:
coord_df.to_csv("/content/drive/My Drive/Colab Data/ebvc_routes/PowerPlants_with_Coordinates.csv", index=False)